In [31]:
# Running for the first time? Install Keras:
#!pip install keras
#!pip install tensorflow-gpu --yes
#!pip install opencv-python

In [74]:
import sklearn
import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import pandas as pd
from tensorflow.keras.applications.resnet50 import ResNet50
#from tensorflow.keras.applications.inception_v3 import InceptionV3
# MobileNet?
import cv2
from PIL import Image
import pandas as pd
from os import listdir
from os.path import isfile, join

In [75]:
## Find name of 
import requests
def return_name(wnid):
    name = 'http://www.image-net.org/api/text/wordnet.synset.getwords?wnid='+wnid
    eng_name = requests.get(name).content.decode("utf-8").strip('\n')
    return eng_name

In [76]:
def read_process_image(path, transformation):
    """
    Reads an image and performs the desired transformation
    """
    img = cv2.imread(path,cv2.IMREAD_COLOR)
    
    if img is None:
        return None, None
    rows,cols, channels = img.shape

    if transformation == 'translate':
        x_translate = 1
        y_translate = 1
        M = np.float32([[1,0,x_translate],[0,1,y_translate]]) # translation
        res = cv2.warpAffine(img,M,(cols,rows))
        
    if transformation == 'scale':
        scale_x = 1 # Reduce x by 1 pixel
        scale_y = 1
        res = cv2.resize(img,(cols - scale_x, rows - scale_y), interpolation = cv2.INTER_LINEAR)
        
    if transformation == 'rotate_180': # Rotates 180 deg
        res = cv2.rotate(img, cv2.ROTATE_180)
        
    if transformation == 'rotate_90': # Rotates 90 deg
        res = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        
    if transformation == 'horizontal_flip':
        res = cv2.flip(img, 1)


    res = cv2.cvtColor(res, cv2.COLOR_BGR2RGB)
    res = Image.fromarray(res)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(img)
    return img, res

In [77]:
def preds_comparison(pred1, pred2):
    """
    Calculates metrics based on two sets of predictions.
    """
    k = len(pred1)
    if k < 1:
        return
    if pred1[0][0] == pred2[0][0]:
        top1_changed = 0
    else:
        top1_changed = 1
    top1_class = pred1[0][0]
    pred_2_value = 0
    for pred in pred2:
        if pred[0] == top1_class:
            pred_2_value = pred[2]
    
    top1_delta = abs(pred1[0][2] - pred_2_value)
    
    avg_delta = abs(sum([abs(pred1[i][2] - pred2[i][2]) for i in range(k)]) / k)
    return top1_changed, top1_delta, avg_delta

In [88]:
def predict(model, img, img_transformed):
    """
    Given a model, an image and a transformed image, generate the predictions
    """
    k = 3
    # Original
    x = np.expand_dims(img, axis=0)
    x = preprocess_input(x)

    preds = model.predict(x)
    #print('Original:', decode_predictions(preds, top=10)[0])
    pred_orig = decode_predictions(preds, top=k)[0]
    top1_class_orig_pred = pred_orig[0][1]
    
    # Transformed 
    x = np.expand_dims(img_transformed, axis=0)
    x = preprocess_input(x)

    preds = model.predict(x)
    #print('Transformed:', decode_predictions(preds, top=k)[0])
    pred_trans = decode_predictions(preds, top=k)[0]
    top1_class_trans_pred = pred_trans[0][1]
    
    top1_changed, top1_delta, avg_delta = preds_comparison(pred_orig, pred_trans)
    return top1_changed, top1_delta, avg_delta, top1_class_orig_pred, top1_class_trans_pred

In [79]:
model = ResNet50(weights='imagenet')

In [89]:
output_df = pd.DataFrame([],
                   columns=['Class Name', 'Orig Pred Class Name', 'Trans Pred Class Name',
                            'Top 1 Changed', 'Top 1 Delta', 'Avg Delta', 'Path'])



wnid_list = ['n02509815', 'n07747607']
for wnid in wnid_list:
    folder_path = 'images/{}/'.format(wnid)
    class_nm = return_name(wnid)
    image_paths = [folder_path + f for f in listdir(folder_path) if isfile(join(folder_path, f))]
    for image_path in image_paths:
        img, res = read_process_image(image_path, 'translate')
        if img is None:
            continue
        top1_changed, top1_delta, avg_delta, top1_class_orig_pred, top1_class_trans_pred = predict(model, img, res)
        row_append = np.array([class_nm, top1_class_orig_pred, top1_class_trans_pred, top1_changed, top1_delta, avg_delta, image_path])
        output_df = output_df.append(dict(zip(output_df.columns, row_append)), ignore_index=True)
    output_df = output_df.astype(dtype= {"Top 1 Changed":"int64",
            "Top 1 Delta":"float64","Avg Delta":"float64"})

In [90]:
output_df.loc[lambda f: f['Top 1 Changed'] == 1] # All results that changed

,Class Name,Orig Pred Class Name,Trans Pred Class Name,Top 1 Changed,Top 1 Delta,Avg Delta,Path
89,lesser panda\nred panda\npanda\nbear cat\ncat ...,lesser_panda,birdhouse,1,0.026042,0.011703,images/n02509815/180.jpg
345,orange,pole,traffic_light,1,0.002450,0.003718,images/n07747607/117.jpg
355,orange,orange,Granny_Smith,1,0.064665,0.046131,images/n07747607/126.jpg
396,orange,orange,Granny_Smith,1,0.064665,0.046131,images/n07747607/163.jpg
443,orange,orange,lemon,1,0.052683,0.027471,images/n07747607/205.jpg
455,orange,lemon,orange,1,0.095925,0.060260,images/n07747607/216.jpg
482,orange,strawberry,lemon,1,0.053144,0.023803,images/n07747607/240.jpg
551,orange,maraca,orange,1,0.004549,0.003812,images/n07747607/302.jpg
565,orange,orange,lemon,1,0.008286,0.006653,images/n07747607/315.jpg
622,orange,pencil_sharpener,web_site,1,0.041601,0.004647,images/n07747607/367.jpg


In [91]:
output_df # All results

,Class Name,Orig Pred Class Name,Trans Pred Class Name,Top 1 Changed,Top 1 Delta,Avg Delta,Path
0,lesser panda\nred panda\npanda\nbear cat\ncat ...,lesser_panda,lesser_panda,0,0.004891,0.003051,images/n02509815/1.jpg
1,lesser panda\nred panda\npanda\nbear cat\ncat ...,lesser_panda,lesser_panda,0,0.101622,0.037882,images/n02509815/10.jpg
2,lesser panda\nred panda\npanda\nbear cat\ncat ...,lesser_panda,lesser_panda,0,0.027945,0.010994,images/n02509815/100.jpg
3,lesser panda\nred panda\npanda\nbear cat\ncat ...,lesser_panda,lesser_panda,0,0.006031,0.002929,images/n02509815/101.jpg
4,lesser panda\nred panda\npanda\nbear cat\ncat ...,lesser_panda,lesser_panda,0,0.093018,0.035851,images/n02509815/102.jpg
...,...,...,...,...,...,...,...
709,orange,orange,orange,0,0.001464,0.001389,images/n07747607/95.jpg
710,orange,orange,orange,0,0.043321,0.017079,images/n07747607/96.jpg
711,orange,orange,orange,0,0.066328,0.026491,images/n07747607/97.jpg
712,orange,orange,orange,0,0.077200,0.033099,images/n07747607/98.jpg


In [92]:
output_df.groupby('Class Name').mean()

,Top 1 Changed,Top 1 Delta,Avg Delta
Class Name,,,
lesser panda\nred panda\npanda\nbear cat\ncat bear\nAilurus fulgens,0.003077,0.037413,0.015412
orange,0.033419,0.031388,0.017757


In [93]:
output_df.groupby('Orig Pred Class Name').count().sort_values('Class Name', ascending=False)

,Class Name,Trans Pred Class Name,Top 1 Changed,Top 1 Delta,Avg Delta,Path
Orig Pred Class Name,,,,,,
orange,336,336,336,336,336,336
lesser_panda,316,316,316,316,316,316
lemon,14,14,14,14,14,14
banana,10,10,10,10,10,10
grocery_store,3,3,3,3,3,3
howler_monkey,2,2,2,2,2,2
strawberry,2,2,2,2,2,2
pomegranate,2,2,2,2,2,2
pineapple,2,2,2,2,2,2


In [48]:
# Selected classes
# n02219486: Ants

In [ ]:
# 3) n07687789; Cornbread